<a href="https://colab.research.google.com/github/highpath/Deep_Learning_Study/blob/master/Kaggle/1_Titanic/Titanic_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titanic Practice

Kaggle에서 가장 기본적인 대회라는 타이타닉 생존자 예측을 연습해보겠습니다.

## Google Drive Mount

구글 드라이브를 코랩에 연동시켜줍니다.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## Import Data
구글드라이브에 업로드 되어 있는 데이터를 읽어옵니다.

In [ ]:
import pandas as pd

In [ ]:
train_data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Deep_Learning_study/Kaggle/1_Titanic/train.csv')
test_data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Deep_Learning_study/Kaggle/1_Titanic/test.csv')

In [ ]:
PassengerId = test_data["PassengerId"]

## EDA(Exploratory Data Analysis)

캐글 사이트에 데이터에 대한 설명이 있습니다.

### Data Dictionary
- Survival : Survival (0=No, 1=Yes)
- Pclass : Ticket class (1=1st, 2=2nd, 3=3rd)
- Sex : Sex
- Age : Age in years
- Sibsp : # of siblings / spouses aboard the Titanic
- Parch : # of parents / childeren aboard the Titanic
- Ticket : Ticket number
- Fare : Passenger fare
- Cabin : Cabin number
- Embarked : Port of Embarkation (C=Cherbourg, Q=Queenstown, S=Southampton)

In [ ]:
train_data.columns, test_data.columns

train_data와 test_data의 column 비교 결과 test_data에는 Survival이 없다는 것을 제외하고는 동일한 것을 알 수 있습니다. 저희는 train_data를 가지고 학습을 시킨 후 test_data의 정보를 보고 승객들의 생존 유무를 예측하겠습니다.

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
pd.set_option('display.max_columns', None)
train_data.head()

In [ ]:
test_data.head()

PassengerId, Name, Ticket 값은 거의 승객 각각의 고유값입니다. 즉, 승객의 생존 유무와는 상관 없는 값이기 때문에 지워야합니다. 그전에

In [ ]:
print(set(train_data["Name"])&set(test_data["Name"]))

In [ ]:
train_data[train_data["Name"]=='Kelly, Mr. James']

In [ ]:
test_data[test_data["Name"]=='Kelly, Mr. James']

혹시나 이름이 같은 승객이 train_data와 test_data에 있나 해서 찾아봤더니 2명이 있었습니다. 그래서 확인해보니 이름 말고는 정보가 다 다른 동명이인이었습니다... 이제는 진짜 PassengerId, Ticket, Name column을 빼주겠습니다.

In [ ]:
train_data.drop(['PassengerId','Ticket','Name'], inplace=True, axis='columns')
test_data.drop(['PassengerId','Ticket','Name'], inplace=True, axis='columns')

## FE(Feature Engineering)


In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
print(train_data["Survived"][train_data["Cabin"].isna()].mean())
train_data["Survived"][train_data["Cabin"].notna()].mean()

Cabin의 유뮤에 따라 생존한 사람 수의 평균이 다른 것을 알 수 있습니다. 따라서 Cabin 데이터 전체를 버리지 않고 Cabin의 유무 값으로 바꾸어주겠습니다.

In [ ]:
train_iscabin = train_data["Cabin"].isna().astype(int)
test_iscabin = test_data["Cabin"].isna().astype(int)

train_data.drop(["Cabin"], inplace=True, axis=1)
test_data.drop(["Cabin"], inplace=True, axis=1)

train_data = train_data.join(train_iscabin)
test_data = test_data.join(test_iscabin)

train_data.rename(columns={'Cabin':'Cabin_OX'}, inplace=True)
test_data.rename(columns={'Cabin':'Cabin_OX'}, inplace=True)

## Split Data

이제 대부분의 분석이 끝났으니 결과값인 Survived column과 나머지를 분리해주겠습니다.

In [ ]:
X_train = train_data.drop(["Survived"], axis=1)
Y_train = train_data["Survived"]
X_test = test_data

In [ ]:
X_train

## Null Value Management

Null 값들을 처리해주겠습니다.

위에서 봤듯이 적절하게 null값을 다른 값으로 채우기 위해선 데이터를 시각화하고 이해하는 능력이 있어야 합니다. 그래야 적절한 방법으로 다른 값을 채울 수 있기 때문입니다. 하지만 저는 그런 능력이 아직 없기에 가장 기본적인 SimpleImputer를 사용하였습니다.

SimpleImputer는 Imputer중 가장 기초적인 방법입니다. Null 값들은 제공된 상수나 Null값이 존재하는 column에서의 통계적인 방법을 통해서 impute됩니다.

impute하기 전에 column을 data type이 object인 것과 아닌 것으로 구분해주겠습니다. impute하는 과정에서 data type이 섞일 수 있기 때문에 따로 구분해서 해주어야 하기 때문입니다.

In [ ]:
s = X_train.dtypes=='object'
categorical_cols = list(s[s].index)

d = X_train.dtypes!='object'
numerical_cols = list(d[d].index)

print(categorical_cols)
print(numerical_cols)

data type이 object인 column은 "most_frequent"방법으로 impute 해주고 나머지는 "median"방법으로 impute 해주겠습니다.

In [ ]:
from sklearn.impute import SimpleImputer

nm_imputer_n = SimpleImputer(strategy='median')
numerical_X_train = pd.DataFrame(nm_imputer_n.fit_transform(X_train[numerical_cols]), columns = numerical_cols)
numerical_X_test = pd.DataFrame(nm_imputer_n.transform(X_test[numerical_cols]), columns = numerical_cols)

nm_imputer_c = SimpleImputer(strategy='most_frequent')
categorical_X_train = pd.DataFrame(nm_imputer_c.fit_transform(X_train[categorical_cols]), columns = categorical_cols)
categorical_X_test = pd.DataFrame(nm_imputer_c.transform(X_test[categorical_cols]), columns = categorical_cols)

## One Hot Encoding

데이터 형태가 object인 "Sex" column과 "Embarked" column을 one hot encoding 해주어야 합니다.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(sparse=False)
categorical_X_train = pd.DataFrame(OH_encoder.fit_transform(categorical_X_train))
categorical_X_test = pd.DataFrame(OH_encoder.transform(categorical_X_test))

In [ ]:
X_train = numerical_X_train.join(categorical_X_train)
X_test = numerical_X_test.join(categorical_X_test)

In [ ]:
X_train

## Train


In [ ]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dropout, Dense

In [ ]:
model = Sequential()

model.add(Dense(128, activation='relu', input_shape = (11,)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=keras.losses.BinaryCrossentropy(),
              metrics = [keras.metrics.AUC()])

In [ ]:
history = model.fit(
    X_train,
    Y_train,
    batch_size=4,
    epochs=64
)

## Generate Predictions

In [ ]:
y_preds = model.predict_classes(X_test)

In [ ]:
y_preds

In [ ]:
import os

!cd "/content/gdrive/My Drive/Colab Notebooks/Deep_Learning_study/Kaggle/1_Titanic"

In [ ]:
file_name = "Titanic_practice.csv"

y_pred_series = pd.Series(y_preds.flatten(), name='Survived')

file = pd.concat([PassengerId, y_pred_series], axis=1)

file.to_csv(file_name, index=False)